# Facial recognition and trigger

Installing necessary libraries

In [1]:
pip install opencv-python opencv-contrib-python pillow numpy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


import necessary libraries

In [2]:
import cv2
import numpy as np
from utils import get_faces_lib, save_to_disk, get_images_and_labels, get_camera


creating necessary folders

In [3]:
mkdir -p FacialRecognitionProject dataset trainer

## Registering a face

In [4]:
def register_face():
    """
    register a face and save 30 images of the face to disk
    """
    cam = get_camera()
    
    face_detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    # For each person, enter one numeric face id
    face_id = input("\n enter user id and press <enter> ==>  ")
    name = input("\n enter user's name and press <enter> ==>  ")
    print("\n [INFO] Initializing face capture. Look the camera and wait ...")
    # Initialize individual sampling face count
    count = 0
    while True:
        ret, img = cam.read()
        img = cv2.flip(img, 1)  # flip video image vertically
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_detector.detectMultiScale(gray, 1.3, 5)

        for (x, y, w, h) in faces:
            cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
            count += 1
            # Save the captured image into the datasets folder
            cv2.imwrite(
                f"dataset/User.{face_id}.{count}.jpg",
                gray[y : y + h, x : x + w],
            )
            cv2.imshow("image", img)

        k = cv2.waitKey(100) & 0xFF  # Press 'ESC' for exiting video
        if k == 27:
            break
        elif count >= 30:  # Take 30 face sample and stop video
            break
    # Do a bit of cleanup
    print("\n [INFO] Exiting Program and cleanup stuff")
    cam.release()
    cv2.destroyAllWindows()
    # save the name of the person
    save_to_disk({face_id: name})


In [5]:
# register_face()

## Training the model

train our model to recognise the registered faces

In [6]:
# Path for face image database
path = "dataset"
recognizer = cv2.face.LBPHFaceRecognizer_create()

print("\n [INFO] Training faces. It will take a few seconds. Wait ...")

faces, ids = get_images_and_labels(path)
recognizer.train(faces, np.array(ids))
# Save the model into trainer/trainer.yml
recognizer.write("trainer/trainer.yml")
print(f"\n [INFO] {len(np.unique(ids))} faces trained. Exiting Program")



 [INFO] Training faces. It will take a few seconds. Wait ...

 [INFO] 2 faces trained. Exiting Program


In [7]:
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read("trainer/trainer.yml")
faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
font = cv2.FONT_HERSHEY_SIMPLEX

names = get_faces_lib()

# Initialize and start realtime video capture
cam = get_camera()

# Define min window size to be recognized as a face
minW = 0.1 * cam.get(3)
minH = 0.1 * cam.get(4)

while True:
    ret, img = cam.read()
    img = cv2.flip(img, 1)  # Flip vertically
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.2,
        minNeighbors=5,
        minSize=(int(minW), int(minH)),
    )
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        id, confidence = recognizer.predict(gray[y : y + h, x : x + w])

        # If confidence is less them 100 ==> "0" : perfect match
        name = names.get(str(id), "unknown") if confidence < 100 else "unknown"
        confidence = " {:.2f}".format(confidence)
        cv2.putText(img, str(name), (x + 5, y - 5), font, 1, (255, 255, 255), 2)
        cv2.putText(img, str(confidence), (x + 5, y + h - 5), font, 1, (255, 255, 0), 1)

    cv2.imshow("camera", img)
    k = cv2.waitKey(30) & 0xFF
    if k == 27:  # press 'ESC' to quit
        break
